# Warsztaty Python w Data Science

---
## Web Scraping - część 1 z 2  

- ### Budowa web crawlera
 - #### Anatomia pająka
 - #### Zarządzanie granicą (*Crawling Frontier*)
 - #### Jak scrapować etycznie i bezpiecznie
- ### Anatomia strony WWW
 - #### HTML jaki jest - każdy widzi
 - #### Parsowanie pobranych danych
- ### Prosty, praktyczny scraper
---


## Budowa (web) crawlera

*Web crawler, webbot, pająk, spiderm pełzacz, web crawler, web wanderer, scraper* - program zbierający informacje o strukturze i treściach stron WWW. Używany najczęściej do 

### Anatomia pająka


*pająk* - to program który:
- odwiedzają linki ze listy określanej jako granica (*the frontier*)
- z odwiedzonych stron wyciąga i zapisuje informacje
  - w szczególności dalsze linki (web indexing)
  - odpowiednie linki uzupełniają *granicę* crawlingu
  - informację w sposób trwały zapisuje web scraper 


### Zarządzanie granicą (*Crawling Frontier*)

- Zakres granicy powinien być ZAWSZE na początku określony
    - najlepiej z góry zawężony do określonej liczby i typu linków
- Granica winna być mocno ograniczana
- Granica rozbudowywana powinna być BARDZO selektywnie
- Duża granica powoduje problemy skali

---
### Jak scrapować etycznie i bezpiecznie

1. Po pierwsze - nie szkodzić! Nie obciążaj niepotrzebnie strony scrapowanej
2. Przestrzegaj `robots.txt` i warunków korzystania z usługi
3. Miej na uwadze, że bazy danych są chronione na podstawie przepisów ustawy o ochronie baz danych 
4. Przestrzegaj RODO
5. Nie ukrywaj się
6. Gdzie to możliwe,  korzystaj z API
---

# art. 8 ustawy z dnia 27 lipca 2001 r. o ochronie baz danych (Dz.U. Nr 28, poz. 1402 ze zm.) 

1. Wolno korzystać z istotnej, co do jakości lub ilości, części rozpowszechnionej bazy danych:

  1)   do własnego użytku osobistego, ale tylko z zawartości nieelektronicznej bazy danych,

  2)   w charakterze ilustracji, w celach _**dydaktycznych lub badawczych**_ (podkreslenie moje), ze wskazaniem źródła, jeżeli takie korzystanie jest uzasadnione niekomercyjnym celem, dla którego wykorzystano bazę,
  
  3)   do celów bezpieczeństwa wewnętrznego, postępowania sądowego lub administracyjnego.

2. Nie jest dozwolone powtarzające się i systematyczne pobieranie lub wtórne wykorzystanie sprzeczne z normalnym korzystaniem i powodujące nieusprawiedliwione naruszenie słusznych interesów producenta.
---

### Anatomia strony WWW
#### HTML jaki jest - każdy widzi


In [ ]:
html_doc = """
<html>
<head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
    <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
    <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
    <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
    and they lived at the bottom of a well.</p>

<p class="story">...</p>
</body></html>
"""

---
#### Parsowanie pobranych danych

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

print(soup.prettify())

---
## HTML ma
- strukturę drzewiastą
- znaczniki się zagnieżdzają i mają mieć (w teorii):
  - `<a>` - początek
  - `</a>` - koniec
- pomiędzy początkiem a końcem są tzw. dzieci
- znaczniki nie mogą się "zazębiac" np. `<a><b></a></b>` (w teorii ...)
- znaczniki mają atrybuty - np. `<a href="linkdostrony">Tu jest link</a>
---

In [ ]:
from bs4 import BeautifulSoup
import lxml
soup = BeautifulSoup(html_doc, 'lxml')

In [ ]:
soup.p

In [ ]:
soup.p['class']

In [ ]:
soup.a

### Szukanie po znaczniku

In [ ]:
soup.find_all('a')

### Szukanie po id

In [ ]:
soup.find(id="link3")

### Wyciąganie atrybutów

In [ ]:
[ link.get('href') for link in soup.find_all('a')]

### Wędrowanie po drzewie

In [ ]:
soup.a

In [ ]:
soup.a.find_next_sibling("a")

In [ ]:
soup.p

In [ ]:
soup.p.find_next_sibling("p")

In [ ]:
pn=soup.p.find_next_sibling("p")
children = pn.children

In [ ]:
children

In [ ]:
lista = [ x for x in children ]
lista

In [ ]:
lista[1].get('href')

In [ ]:
head_tag = soup.head
head_tag

In [ ]:
for child in head_tag.children:
    print(child)

In [ ]:
for child in head_tag.descendants:
    print(child)

In [ ]:
last_a_tag = soup.find("a", id="link3")
last_a_tag


In [ ]:
last_a_tag.next_sibling

In [ ]:
last_a_tag.next_element

In [ ]:
last_a_tag.parent

### Szukanie przy użyciu predykatu (funckji logicznej)

In [ ]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')

soup.find_all(has_class_but_no_id)

In [ ]:
soup.find_all(id='link2')

In [ ]:
soup.find_all("a", class_="sister")

In [ ]:
soup.find_all("a")
soup("a")

---
## Prosty, praktyczny scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


frontier = [ f'https://www.gumtree.pl/s-mieszkania-i-domy-do-wynajecia/warszawa/page-{n}/v1c9008l3200008p{n}' for n in range(3) ]
data = {'title': [], 'link':[]}
    
for url in frontier:
    time.sleep(1)
    page = requests.get(url)
    print (page)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


frontier = [ f'https://www.gumtree.pl/s-mieszkania-i-domy-do-wynajecia/warszawa/page-{n}/v1c9008l3200008p{n}' for n in range(3) ]
data = {'title': [], 'link':[]}
    
for url in frontier:
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    titles = [flat.next_element for flat in soup.find_all('a', class_ = "href-link tile-title-text")] 
    print(titles)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

frontier = [ f'https://www.gumtree.pl/s-mieszkania-i-domy-do-wynajecia/warszawa/page-{n}/v1c9008l3200008p{n}' for n in range(3) ]
data = {'title': [], 'link':[]}
    
for url in frontier:
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    titles = [flat.next_element for flat in soup.find_all('a', class_ = "href-link tile-title-text")] 
    links = ['https://www.gumtree.pl' + link.get('href')
                for link in soup.find_all('a', class_ ="href-link tile-title-text")]
    print(links)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

frontier = [ f'https://www.gumtree.pl/s-mieszkania-i-domy-do-wynajecia/warszawa/page-{n}/v1c9008l3200008p{n}' for n in range(3) ]
data = {'title': [], 'link':[]}
    
for url in frontier:
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    titles = [flat.next_element for flat in soup.find_all('a', class_ = "href-link tile-title-text")] 
    links = ['https://www.gumtree.pl' + link.get('href')
                for link in soup.find_all('a', class_ ="href-link tile-title-text")]
    data['link'].extend(links)
    data['title'].extend(titles)
print(data)

In [ ]:
import pandas as pd
df = pd.DataFrame(data).drop_duplicates()
df.head(100)

In [ ]:
df.to_csv("./gumtree_all_pages.csv", sep=';',index=False, encoding = 'utf-8')

---
# Zadanie 1.
Wyciągnąć z dowolnego ogłoszenia cenę mieszkania

# Zadanie 2.
Wyciągnąć z dowolnego ogłoszenia cenę ZA METR
